In [ ]:
#importing all necessary libraries
import pandas as pd
from utils import *
from tifffile import imread, imsave
from skimage import exposure
from scipy.ndimage import median_filter
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
import imageio
from base64 import b64encode
from IPython.display import HTML
import caiman as cm
from caiman.motion_correction import MotionCorrect, tile_and_correct, motion_correction_piecewise
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from cellpose import models
from tqdm import tqdm
from tifffile import imread, imsave
from statsmodels.stats.multitest import multipletests
from scipy import stats as st
from skimage.morphology import dilation
from pystackreg import StackReg
import pystackreg
from skimage import io

In [ ]:
#@figure settings
mpl.rcParams.update({
    'figure.subplot.wspace': .1,
    'figure.subplot.hspace': .1,
    'figure.figsize': (18, 13),
    'svg.fonttype':'none'
})

In [ ]:
#file loading setting
frame_function = get_cycle # the function for extracting the frame
z_function = get_end# the function for extracting the z level
z = 'z1' # the z level of interest for this analysis
f_ch = 'ch2' # the functional channel 
mc_ch = 'ch1' # the channel to use for motion correction
ds = False # whether or not to downsample the movie spatially
step_size = 2 # number of pixels to skip when downsampling
remake = False # whether or not to remake the downsampled and cropped movies
frame_period = 0.585 # time in seconds between frames, get this from the xml file

# for the videos
ds_ratio = 0.5 # the fraction of frames to show in the video
speed_up = 20  # how much to speed up the video by

##############################################################################################
fr = 1/frame_period # frame rate
fps = speed_up * fr * ds_ratio # frames per second for writing the video files

In [ ]:
# for cropping and concatenating multiple files
from collections import OrderedDict
multi_path = [Path('Path').resolve(), 
              Path('Path').resolve()]
multi_crop = True
multi_crop_start = [ , , ]
multi_crop_len = [ , , ]
multi_sp = {}
concat_filenames = {}
ch_list = [mc_ch,f_ch]
multi_data = {}

for ch in ch_list:
    multi_data[ch] = OrderedDict()
    for count, session in enumerate(multi_path):
        print("session "+str(count))
        print(session.as_posix())
        multi_sp[count]  = combine_tiffs(session, get_frame_cycle = frame_function, get_z = z_function)
        if multi_crop:
                p = multi_sp[count][z][ch]
                data = imread(p).squeeze()
                multi_data[ch][count] = data[multi_crop_start[count]:multi_crop_start[count] + multi_crop_len[count], :, :]                

# Optional step to perform PyStackReg

In [ ]:
# This cell iterates over the sessions that were defined in multi_path above.
# For each session, a reference image is generated by averating the middle 50% of frames.
# The reference image is appended to the beginning of the stack and the StackReg affine motion correction
# runs using this images as the reference.
# Then we apply the transform discovered with the motion correction channel to the mc channel and the functional channel
# Then we chop the first frame back off and we ought to have affine motion corrected tiffs for both channels for all sessions.

out_mcch = {}
out_fch = {}

affine_data = {}
for ch in ch_list:
    affine_data[ch] = OrderedDict()

for count, session in enumerate(multi_path):
    refsize = np.ceil(np.size(multi_data[mc_ch][count],0)/2)
    # create 'ref' which is the mean of the middle 50% of images (approximately)
    ref = np.mean(multi_data[mc_ch][count][int(np.ceil(refsize/2)):int(np.ceil(refsize/2)+refsize),:,:],0)
    toreg_mc = multi_data[mc_ch][count]
    toreg_fc = multi_data[f_ch][count]
    firstframetrick_mc = np.vstack([ref[np.newaxis,...], toreg_mc]) # have to put ref as first frame because sr will not accept a reference image I created
    firstframetrick_fc = np.vstack([ref[np.newaxis,...], toreg_fc]) # have to put ref as first frame because sr will not accept a reference image I created

    #Affine transformation
    sr = StackReg(StackReg.AFFINE)
    out_aff = sr.register_stack(firstframetrick_mc, reference='first')
    out_mcch[count] = sr.transform_stack(firstframetrick_mc)
    out_fch[count] = sr.transform_stack(firstframetrick_fc)
    
    affine_data[mc_ch][count] = out_mcch[count][1:,:,:]
    affine_data[f_ch][count] = out_fch[count][1:,:,:]

In [ ]:
sp = {}

# This cell takes the affine transformed data and concatenates it all into one long movie across all sessions.
for ch in ch_list:
    concat_data = np.concatenate(list(affine_data[ch].values()))
    p = multi_sp[count][z][ch]
    fname = Path(p).parents[2] / Path(Path(p).parts[-2]+'_concat') / 'concat_cropped_affine.ome.tif'
    mov_dir = Path(fname.parent)
    concat_filenames[ch] = fname
    if not mov_dir.exists(): mov_dir.mkdir(parents = True)
    imsave(fname, concat_data)
sp[z] = concat_filenames

# Rigid Motion Correction

In [ ]:
# settings for motion correction
mc_params = {
    "max_shifts": (50, 50),  # maximum allowed rigid shift in pixels (view the movie to get a sense of motion)
    "niter_rig" :  3,          # number of times to perform rigid registration (i'd recommend doing at least 2 especially for these shorter recordings) # number of chunks for parallel processing ()
    "pw_rigid"  : True,       # flag for performing rigid or piecewise rigid motion correction (false for now as we want to asses rigid registration first)
    "shifts_opencv": True ,    # flag for correcting motion using bicubic interpolation (otherwise FFT interpolation is used)
    "border_nan"   : 'copy',
    "nonneg_movie" : True,
    "use_cuda"  : False,
    "strides"   : (64, 64), # size of patches for nonrigid motion correction in pixels
    "overlaps"  : (32, 32), # number of pixrls of overlap between patches
    "max_deviation_rigid"  : 3, # maximum allowed deviation of any individual patch's registered shift from the rigid shift
    "upsample_factor_grid" : 4,
}

In [ ]:
#visualize what different patch sizes will sample
raw2 = cm.load_movie_chain([sp[z][mc_ch]])
i,j=2,2 #rough patch index (ignoring overlaps)
x, y = mc_params['strides'] # stride
_,ax = plt.subplots(1,1)
ax.imshow(raw2[0, i*y:(i+1)*y + 1, j*x:(j + 1)*x + 1]);
del raw2

In [ ]:
#setting up clusters for parallel processing
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)
#create motion correction object and perform rigid motion correction
mc = MotionCorrect([sp[z][mc_ch]], dview=dview, **mc_params)
mc.motion_correct(save_movie=True)

In [ ]:
#@title plot shifts
plt.plot(np.array(mc.shifts_rig)[:,0], label = 'x shifts');
plt.plot(np.array(mc.shifts_rig)[:,1], label = 'y shifts');
plt.legend();
plt.xlabel('frames');
plt.ylabel('pixels');

In [ ]:
if mc.pw_rigid:
    _,ax=plt.subplots(2,1)
    ax[0].plot(np.array(mc.x_shifts_els) - np.array(mc.shifts_rig)[:,0,None]);
    ax[0].set_title('x shift deviations');
    ax[1].plot(np.array(mc.y_shifts_els) - np.array(mc.shifts_rig)[:,1,None]);
    ax[1].set_title('y shift deviations');

In [ ]:
corr2 = np.array(cm.load(mc.mmap_file))
raw2 = np.array(cm.load_movie_chain([sp[z][mc_ch]]))

c = corr2.reshape(corr2.shape[0], -1)
r = raw2.reshape(raw2.shape[0], -1)

def corr_m(x):
    xm = x.mean(axis=0)
    xm_ = xm - xm.mean()
    x_ = x - x.mean(axis=1)[:,None]
    corr = (x_ @ xm_) / np.sqrt((x_ @ x_.T).diagonal() * (xm_**2).sum())
    return corr

c = corr_m(c)
r = corr_m(r)

fig, ax = plt.subplots(1,2, figsize = (10,5))
ax[0].plot(c, label = 'corrected')
ax[0].plot(r, label = 'raw')
ax[0].set(xlabel = 'Frame', ylabel = 'Correlation with Mean')
ax[0].legend()

ax[1].scatter(x = r, y= c)
xlim = ax[1].get_xlim()
ylim = ax[1].get_ylim()
nlim = ( min(xlim[0], ylim[0]), max(xlim[1], ylim[1]) )
ax[1].plot(nlim, nlim, color = 'k', ls = '--')
ax[1].set( ylabel = 'Corrected', xlabel = 'Raw', 
           title = 'Correlation with Mean', 
           xlim = nlim, ylim = nlim)

sns.despine()
fig.tight_layout(pad = 1.)

del corr2
del raw2

In [ ]:
# clear up some memory
if 'c' in locals():del c
if 'cl' in locals(): del c1
if 'c2' in locals(): del c2
if 'data_url' in locals(): del data_url
if 'display_movie' in locals(): del display_movie
if 'max_' in locals(): del max_
if 'min_' in locals(): del min_
if 'mp4' in locals(): del mp4
if 'path' in locals():del path

In [ ]:
# save memory map file for structural channel
border_to_0 = 0 if mc.border_nan == 'copy' else mc.border_to_0 
fname_new = cm.save_memmap(mc.mmap_file, base_name='memmap_', order='C',
                           border_to_0=border_to_0, dview=dview)

# apply the shifts to the functional channel and save
fname_new2 = mc.apply_shifts_movie([sp[z][f_ch]], save_memmap=True, order = 'C',
                                   save_base_name=(Path(sp[z][f_ch]).resolve().parent/'memmap_').as_posix())

In [ ]:
# movie of just corrected functional channel

anchor = ( , ) # where to anchor the stim bar
wd, ht = , # width and height of the stim bar
t_stim_st =  # time in seconds of the start of the stim
t_stim_end =  # time in seconds of the end of the stim

###########################################################
fr_stim_st = int(ds_ratio * t_stim_st)
fr_stim_end = int(ds_ratio * t_stim_end)

mv = cm.load(fname_new2).resize(1, 1, ds_ratio)
_min, _max = np.percentile(mv, 0.01), np.percentile(mv, 99.99)
mv = np.array(255 * (mv - _min)/(1.1 * (_max - _min)), dtype = 'uint8')
mv = mv[:,:,:,None] * np.ones(mv.shape + (3,))
mv[fr_stim_st:fr_stim_end, anchor[0]:anchor[0] + ht, anchor[1]:anchor[1] + wd, 0] = 255
imageio.mimwrite((Path(sp[z][mc_ch]).parent/f'pre_processed_{speed_up}x_speed.mp4').as_posix(),
                 mv.astype('uint8'), fps = fps,  quality=7)
del mv

mp4 = open((Path(sp[z][mc_ch]).parent/f'pre_processed_{speed_up}x_speed.mp4').as_posix(),'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
print(fname_new)
print(fname_new2)

# Source Extraction

In [ ]:
#getting data

# fname_new = '/Users/nathanielnyema/Downloads/tmp_ap_2pdata_storage/AP -2-070/ch1_z_1_movie/memmap__d1_512_d2_512_d3_1_order_C_frames_300_.mmap'
# fname_new2 = '/Users/nathanielnyema/Downloads/tmp_ap_2pdata_storage/AP -2-070/ch2_z_1_movie/memmap__d1_512_d2_512_d3_1_order_C_frames_300_.mmap'

Yr, dims, T = cm.load_memmap(fname_new)
images = np.reshape(Yr.T, [T] + list(dims), order='F')

Yr, dims, T = cm.load_memmap(fname_new2)
images2 = np.reshape(Yr.T, [T] + list(dims), order='F')

del Yr
del dims
del T

In [ ]:
#stop and restart clusters to clear up memory
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

### Use Cellpose to find ROIs to seed the CNMF

In [ ]:
# first compute the local correlation map

method = 'max'
filt = True # whether or not to spatially filter the resulting image
kern = 1 # kernel size of filter

#############################
func = cm.load(fname_new2)

if method == 'max':
    f_lc = np.percentile(func, 99, axis=0)
elif method == 'local_correlation':
    f_lc = func.local_correlations()
elif method == 'mean':
    f_lc = np.mean(func, axis=0)
else:
    print('unrecognized method')
    f_lc = np.zeros(func.shape[1:])
    
if filt:
    f_lc = median_filter(f_lc, kern)
    
plt.figure(figsize = (10,10))
plt.imshow(f_lc)

del func

In [ ]:
channels = [[0,0]]
flow_threshold = 2
cellprob_threshold = -1
diameter = 15
model_type = 'cyto'

model = models.Cellpose(model_type='cyto', gpu=False)
masks, _, _, _ = model.eval(f_lc,  diameter= diameter, channels=channels, 
                            flow_threshold = flow_threshold,
                            cellprob_threshold = cellprob_threshold)
Ain = np.array([(masks==i).flatten('F') for i in np.sort(np.unique(masks))[1:]]).T

#create image with masks for GUI purposes
im = (f_lc*190//f_lc.max()).astype(np.uint8)
im = cv.cvtColor(im,cv.COLOR_GRAY2RGB)
ms = np.zeros([*f_lc.shape,3],dtype=np.uint8)
for i in Ain.T:
    ms[i.reshape(f_lc.shape,order='F')] = np.tile(65*np.random.rand(1,3),(i.sum(),1)).astype(int)

#show the initial estimates from CellPose
fig, ax=plt.subplots(1,2, figsize = (10,5))
ax[0].imshow(im + ms);
ax[1].imshow(im);
fig.tight_layout(pad = 1.)

np.save(Path(sp[z][f_ch]).parent/'Ain.npy', Ain)

### A GUI to add and remove ROIs

In [ ]:
#here we allow the user to decide if they'd like to make edits to the results from cellpose (ie remove or add neurons)
#NOTE: press the esc key to exit the GUI!
while input('Would you like to remove any neurons (y/n)?  ').lower() in ['y','ye','yes']:
    print('Right click on neurons in the viewer that ')
    neurons,_ms=remove_neurons(Ain,im,ms.copy())
    if input("Are you sure you'd like to remove these neurons (y/n)?  ").lower() in ['y','ye','yes']:
        print('deleting...')
        Ain=np.delete(Ain,neurons,1)
        print(neurons)
        ms=_ms


while input('Would you like to add more neurons (y/n)?  ').lower() in ['y','ye','yes']:
    _ms,mask=draw_masks(im,ms.copy(),np.zeros(images.shape[-2:],np.uint8),show_plot=False)
    if input("Are you sure you'd like to add this neuron (y/n)?  ").lower() in ['y','ye','yes']:
        print('adding...')
        Ain=np.concatenate([Ain,mask.reshape((-1,1),order='F')],axis=1)
        ms=_ms

#remove empty masks that were kept
Ain=Ain[:,~(Ain.sum(axis=0)==0)]

np.save(Path(sp[z][f_ch]).parent/'Ain.npy',Ain)
plt.imshow(im+ms);
plt.show()

In [ ]:
ain_file = list(filter( lambda f: 'Ain' in f.name, Path(sp[z][f_ch]).parent.iterdir() ))
if len(ain_file)>0:
    Ain = np.load(ain_file[0])
else:
    print('no masks found!')

### Run CNMF

In [ ]:
import numpy as np
Ain = np.load('/Users/kphuang/Documents/two-photon-analysis/062923 ensure_cinacalcet_LiCl/ensure_cina/ch2_z_2_movie_concat/Ain.npy')
np.shape(Ain)

In [ ]:
#set parameters for source extraction
opts_dict = {
    'fr': fr,
    'decay_time': 1.8, # this should be set to 1.8 for GCamp6s, but can be changed to 0.4 for faster indicators
    'p': 2, # this must be 2 for GCamp6s, it can be set to 1 for faster indicators
    'nb': 2, # the number of background components, 
    'rf': None, #must be None for seeded mode
    'only_init':False, #must be false for seeded mode
    'min_SNR': 2.0,
    'use_cnn': False, # whether or not to use a convolutional neural network to classify rois as good or bad
    'use_cuda':False, # whether or not to use GPUs
    ## the rest of these are necessary parameters to set for that get ignored
    ## since we are seeding the algorithm for initialization
    'K': 300,  
    'gSig':[10,10],
}

opts = params.CNMFParams(params_dict=opts_dict)

In [ ]:
#stop and restart clusters to clear up memory
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

# #do an initial fit of the cnmf
cnm = cnmf.CNMF(n_processes, params=opts, dview=dview, Ain=Ain)
cnm = cnm.fit(images2)

In [ ]:
#view the initial contours to see if we need to change any parameters
Cn = f_lc
cnm.estimates.plot_contours(img=Cn);

In [ ]:
# Visualize raw traces
cnm.estimates.dims=cnm.dims
cnm.estimates.view_components(img=Cn,idx=[41]);

### Save the cnmf object for later analyses

In [ ]:
cnm.save((Path(sp[z][f_ch]).parent/'cnmf.hdf5').as_posix())

### Potional step to compute  Δ𝐹/𝐹 here 

In [ ]:
def custom_df_f(c, baseline, quantileMin = 50, use_residuals = False):
    """
    A custom version of the detrend_df_f function in the original CaImAn code.
    In this version, we compute the baseline fluorescence using only a specified number
    of frames at the beginning of the recording. By setting quantileMin to 50 we effectively
    normalize to the median of the baseline period.
    """
    A, C, b, f, YrA = c.estimates.A, c.estimates.C, c.estimates.b, c.estimates.f, c.estimates.YrA
    F = C + YrA if use_residuals else C
    F = F * np.sqrt((A.T @ A).diagonal()[:,None])
    B = b @ f
    f0 =  F + (A.T @ B)
    f0 = np.percentile(f0[:,:baseline], quantileMin, axis=1)
    fb = np.percentile(F[:,:baseline], quantileMin, axis=1)
    df_f = (F - fb[:,None])/f0[:,None]
    
    return df_f


bline = 60
fl_acc = custom_df_f(cnm, bline, use_residuals = False)

### Optional step to create a denoised movie of the denoised 

In [ ]:
#create a denoised movie of the denoised 
mv = np.rollaxis((cnm.estimates.A @ cnm.estimates.C).reshape((*cnm.dims,-1), order = 'F'), -1)
_min, _max = mv.min(), mv.max()
mv = np.array(255* (mv-_min)/(_max-_min), dtype = 'uint8')
imageio.mimwrite((Path(sp[z][f_ch]).parent/'denoised.mp4').as_posix(), mv, fps = 30,  quality=7)

# Manually separate subregions and calculate Z-score

In [ ]:
import bokeh.plotting as bpl
from bokeh.layouts import row
from bokeh.plotting import show
from bokeh.models import LassoSelectTool

import matplotlib.pyplot as plt
import numpy as np
from caiman.source_extraction.cnmf import cnmf as cnmf

try:
    if __IPYTHON__:
        # this is used for debugging puqrposes only. allows to reload classes
        # when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

from caiman.utils.visualization import plot_contours, nb_view_patches, nb_pick_dots, nb_show_work, get_contours
bpl.output_notebook()

In [ ]:
# loading hdf5 files and stacked images
cnm2 = cnmf.load_CNMF('Path')       # load hdf5 file
img = plt.imread('Path')            # load stacked image

### Use lasso on left plot to select analysis region 1 (Area Postrema)

In [ ]:
area_indices = [None, None]
p1 = nb_pick_dots(img, cnm2.estimates.A, cnm2.estimates.dims[0], cnm2.estimates.dims[1], bg_brightness=0.8, line_width=1, show=True, dot_color='dodgerblue')

In [ ]:
area_indices[0] = selected_indices

### Use lasso on left plot to select analysis region 2 (Nucleus of Solitary Tract)

In [ ]:
p1 = nb_pick_dots(img, cnm2.estimates.A, cnm2.estimates.dims[0], cnm2.estimates.dims[1], bg_brightness=0.4, line_width=1, show=True, dot_color='dodgerblue')

In [ ]:
area_indices[1] = selected_indices

In [ ]:
# Check your work
# All contours in white
pcheck = nb_show_work(img, cnm2.estimates.A, cnm2.estimates.dims[0], cnm2.estimates.dims[1], bg_brightness=0.6, line_color='white', show=False)

# Area 1 plotted in red
coors = get_contours(cnm2.estimates.A[:,area_indices[0]], np.shape(img), thr=None, thr_method='max')
cc1 = [np.clip(cor['coordinates'][1:-1, 0], 0, cnm2.estimates.dims[1]) for cor in coors]
cc2 = [np.clip(cor['coordinates'][1:-1, 1], 0, cnm2.estimates.dims[0]) for cor in coors]
pcheck.patches(cc1, cc2, alpha=1, color=None, line_color='red', line_width=1.5)

# Area 2 plotted in yellow
coors = get_contours(cnm2.estimates.A[:,area_indices[1]], np.shape(img), thr=None, thr_method='max')
cc1 = [np.clip(cor['coordinates'][1:-1, 0], 0, cnm2.estimates.dims[1]) for cor in coors]
cc2 = [np.clip(cor['coordinates'][1:-1, 1], 0, cnm2.estimates.dims[0]) for cor in coors]
pcheck.patches(cc1, cc2, alpha=1, color=None, line_color='yellow', line_width=1.5)
bpl.show(pcheck)

### Calculate dF or Zscore based on specific range for STIM

In [ ]:
def custom_df_f_startend(c, baseline_start, baseline_end, method = 'zscore', use_residuals = False):
    """
    A custom version of the detrend_df_f function in the original CaImAn code.
    In this version, we compute the baseline fluorescence using only a specified number
    of frames at the beginning of the recording. By setting quantileMin to 50 we effectively
    normalize to the median of the baseline period.
    """
    A, C, b, f, YrA = c.estimates.A, c.estimates.C, c.estimates.b, c.estimates.f, c.estimates.YrA
    F = C + YrA if use_residuals else C
    F = F * np.sqrt((A.T @ A).diagonal()[:,None])
    B = b @ f
    f0 =  F + (A.T @ B)
    if method == 'norm_to_median':
        f0 = np.percentile(f0[:,baseline_start:baseline_end], 50, axis=1)
        fb = np.percentile(F[:,baseline_start:baseline_end], 50, axis=1)
    elif method == 'zscore':
        fb = np.mean(F[:,baseline_start:baseline_end], axis=1)
        f0 = np.std(f0[:,baseline_start:baseline_end], axis=1)
    df_f = (F - fb[:,None])/f0[:,None]
    
    return df_f

method = 'zscore'
use_residuals = True
bline_start =     # Enter the baseline_start for this stimulation
bline_end =       # Enter the baseline_end for this stimulation
fl_acc = custom_df_f_startend(cnm2, bline_start, bline_end, method = method, 
                     use_residuals = use_residuals )

In [ ]:
stim1_start =     # Enter the stim_start for this stimulation
stim1_end =       # Enter the stim_end for this stimulation

stim1_area1_data = fl_acc[area_indices[0],stim1_start:stim1_end]
stim1_area2_data = fl_acc[area_indices[1],stim1_start:stim1_end]

np.savetxt('stim1_AP.csv', stim1_area1_data, delimiter=',')
np.savetxt('stim1_NTS.csv', stim1_area2_data, delimiter=',')

# Optional Step to calculate difference betweem stimuli and create colormap

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.base.rois import com

In [ ]:
def custom_df_f_startend(c, baseline_start, baseline_end, method = 'zscore', use_residuals = False):
    """
    A custom version of the detrend_df_f function in the original CaImAn code.
    In this version, we compute the baseline fluorescence using only a specified number
    of frames at the beginning of the recording. By setting quantileMin to 50 we effectively
    normalize to the median of the baseline period.
    """
    A, C, b, f, YrA = c.estimates.A, c.estimates.C, c.estimates.b, c.estimates.f, c.estimates.YrA
    F = C + YrA if use_residuals else C
    F = F * np.sqrt((A.T @ A).diagonal()[:,None])
    B = b @ f
    f0 =  F + (A.T @ B)
    if method == 'norm_to_median':
        f0 = np.percentile(f0[:,baseline_start:baseline_end], 50, axis=1)
        fb = np.percentile(F[:,baseline_start:baseline_end], 50, axis=1)
    elif method == 'zscore':
        fb = np.mean(F[:,baseline_start:baseline_end], axis=1)
        f0 = np.std(f0[:,baseline_start:baseline_end], axis=1)
    df_f = (F - fb[:,None])/f0[:,None]
    
    return df_f

In [ ]:
cnm2 = cnmf.load_CNMF('Path')         # Load hdf5 file
img = plt.imread('Path')              # Load stacked image
center = com(cnm2.estimates.A, cnm2.estimates.dims[0], cnm2.estimates.dims[1])

In [ ]:
#Define the baseline and stimulation time for each stimulation
# STIM 1:
baseline_1 = [ , ]
stim_1 = [ , ]

# STIM 2:
baseline_2 = [ , ]
stim_2 = [ , ]

In [ ]:
method = 'zscore'
use_residuals = True

fl_acc_1 = custom_df_f_startend(cnm2, baseline_1[0], baseline_1[1], method = method, 
                     use_residuals = use_residuals )

stim_mdn_1 = np.median(fl_acc_1[:,stim_1[0]:stim_1[1]], axis=1)

fl_acc_2 = custom_df_f_startend(cnm2, baseline_2[0], baseline_2[1], method = method, 
                     use_residuals = use_residuals )

stim_mdn_2 = np.median(fl_acc_2[:,stim_2[0]:stim_2[1]], axis=1)

# Index.cinacalcet is [Stim 2 - Stim 1]
stim_idx = stim_mdn_2 - stim_mdn_1
stim_idx[np.maximum(stim_mdn_1, stim_mdn_2)<1.64] = np.nan

In [ ]:
plt.imshow(img, cmap='gray',alpha=1)
# you could change edgecolor='none' to edgecolor='black' if you want to try lines around the circles
plt.scatter(center[:,1],center[:,0],c=stim_idx, cmap='bwr', vmin=-10, vmax=10, alpha=1, s=50, edgecolor='none',linewidth=0.75)
plt.colorbar()
fig = plt.gcf()
fig.savefig('cinacalcet.pdf', bbox_inches='tight')